# 一、使用`__slots__`
正常情况下，当我们定义了一个class实例，可以给它绑定任何属性和方法：

In [1]:
class Student(object):
    pass

In [2]:
s = Student()
s.name = 'Michael'
print(s.name)

Michael


还可以尝试给实例绑定一个方法：

In [3]:
# 定义一个函数作为实例方法
def set_age(self, age):
    self.age = age
    
from types import MethodType
# 给实例绑定一个方法
s.set_age = MethodType(set_age, s)
# 调用实例方法
s.set_age(25)
s.age

25

但是这样个实例绑定的方法对另外一个实例是没有用的。

In [4]:
s2 = Student()
s2.set_age(25)

AttributeError: 'Student' object has no attribute 'set_age'

为了给所有的实例都绑定方法，可以给类绑定方法：

In [5]:
def set_score(self, score):
    self.score = score
    
Student.set_score = set_score

In [6]:
s.set_score(100)
s.score

100

In [7]:
s2.set_score(99)
s2.score

99

通常情况下，上面的set_score方法可以直接定义在类中，但是动态绑定允许我们在程序运行过程中给class添加功能。

**使用`__slots__`**<br>
如果我们想要限制实例的属性，比如只允许对`Student`实例添加`name`和`age`属性，可以在定义class的时候，定义一个特殊的`__slots__`变量，来限制能添加的属性：

In [8]:
class Student(object):
    __slots__ = ('name', 'age')

In [9]:
s = Student()
s.name = 'Michael'
s.age = 25

In [10]:
s.score = 99

AttributeError: 'Student' object has no attribute 'score'

由于`score`没有被放到`__slots__`中，所以不能动态绑定。需要注意的是`__slots__`只针对当前类可用，对继承的子类是不起作用的。

# 二、使用`@property`
在绑定属性时，如果我们直接把属性暴露出去，没有办法检查参数，导致属性值被随意修改：

In [11]:
class Student(object):
    pass

def set_score(self, score):
    self.score = score
    
Student.set_score = set_score

s = Student()
s.score = 9999

这样肯定是不行的。为了限制`score`的范围，可以通过一个`set_score()`的方法来设置成绩，并通过`get_score()`来获取成绩，这样就可以进行参数检查：

In [12]:
class Student(object):
    
    def get_score(self):
        return self._score
    
    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100')
        self._score = value

In [13]:
s = Student()
s.set_score(60)
s.get_score()

60

In [14]:
s.set_score(9999)

ValueError: score must between 0 ~ 100

但是上面的调用略显复杂，没有直接使用属性这么简单。为了简化调用，可以使用内置的`@property`装饰器来把方法变成属性调用：

In [17]:
class Student(object):
    
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

把一个getter方法变成属性，只需要加上`@property`就可以了，此时`@property`本身又创建了另一个装饰器`@score.setter`，负责把一个setter方法变成属性赋值，于是我们就有了一个可控的属性操作：

In [18]:
s = Student()
s.score = 60
s.score

60

In [19]:
s.score = 9999

ValueError: score must between 0 ~ 100!

还可以定义只读属性，只定义getter方法，不定义setter方法即可：

In [20]:
class Student(object):
    
    @property
    def birth(self):
        return self._birth
    
    @birth.setter
    def birth(self, value):
        self._birth = value
    
    @property
    def age(self):
        return 2018 - self.birth

上面的`birth`是一个读写属性，但`age`是一个只读属性。

## 练习
请利用`@property`给一个`Screen`对象加上`width`和`height`属性，以及一个只读属性`resolution`：

In [21]:
class Screen(object):
    
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, value):
        if not isinstance(value, int):
            raise ValueError("Screen width must be integer!")
        self._width = value
        
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, value):
        if not isinstance(value, int):
            raise ValueError("Screen height must be integer!")
        self._height = value
        
    @property
    def resolution(self):
        return self._width * self._height

# 测试:
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过!')
else:
    print('测试失败!')

resolution = 786432
测试通过!


# 三、多重继承
继承是面向对象的一个重要方式，通过继承，子类就可以扩展父类的功能。

In [22]:
class Animal(object):
    pass

# 大类
class Mammal(Animal):
    pass

class Bird(Animal):
    pass

# 各种动物
class Dog(Mammal):
    pass

class Bat(Mammal):
    pass

class Parrot(Bird):
    pass

class Ostrich(Bird):
    pass

从上面可以明显看出类之间的继承关系。现在我们要给动物加上`Runnable`和`Flyable`功能，只需要预先定义好两个类：

In [23]:
class Runnable(object):
    def run(self):
        print("Running....")
        
class Flyable(object):
    def fly(self):
        print("Flying....")

对于需要`Runnable`功能的动物，就继承一个`Runnable`，例如`Dog`：

In [24]:
class Dog(Mammal, Runnable):
    pass

In [25]:
class Bat(Mammal, Flyable):
    pass

通过多重继承，一个子类就可以同时获得多个父类的功能。

**MixIn**<br>
在设计类的继承关系时，通常都是单一继承下来的。例如`Ostrich`继承自`Bird`。但是，如果需要混入额外功能，通过多重继承就可以实现，比如让`Ostrich`同时继承`Bird`和`Runnable`。这种设计被称为“MixIn”

In [26]:
class RunnableMixIn(object):
    def run(self):
        print("Running....")
        
class FlyableMixIn(object):
    def fly(self):
        print("Flying....")

class Ostrich(Bird, RunnableMixIn):
    pass

MixIn的目的就是给一个类增加多个功能，这样，在设计类的时候，我们优先考虑通过多重继承来组合多个MixIn的功能，而不是设计多层次的复杂的继承关系。

Python自带的很多库也使用了MixIn，例如`TCPServer`和`UDPServer`，同时服务多个用户就必须使用多线程或者多进程，它们由`ForkingMixIn`和`ThreadingMixIn`提供.

# 四、定制类
类似`__slots__`，Python中还有其他的class有这种功能，可以帮助我们定制类

#### `__str__`

In [27]:
class Student(object):
    def __init__(self, name):
        self.name = name

print(Student('Michael'))

打印出了一堆`<__main__.Student object at 0x7f9bd05c5978>`，输出不是很好看。通过`__str__`方法，可以返回一个比较好看的字符串：

In [28]:
class Student(object):
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return "Student object (name: %s)" % self.name

print(Student('Michael'))

Student object (name: Michael)


In [29]:
# 不使用print，还是不好看
s = Student('Michael')
s

这是因为直接显示变量调用的不是`__str__`，而是`__repr__`，`__str__`返回的是用户看到的字符串，而`__repr__`是返回程序开发者看到的字符串，是用来调试的。解决办法就是再定义一个`__repr__`：

In [30]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return "Student object (name = %s)" % self.name
    __repr__ = __str__

In [31]:
s = Student('Michael')
s

Student object (name = Michael)

#### `__iter__`
如果一个类想被用于`for....in`循环，就必须实现一个`__iter__`方法，该方法返回一个迭代对象，可供Python的for循环调用，使用`__next__`方法不断返回下一个值，直到`StopIteration`

In [32]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1
    def __iter__(self):
        return self       # 实例本身就是迭代的对象，故返回自己
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        if self.a > 100000:
            raise StopIteration()
        return self.a

In [33]:
for n in Fib():
    print(n)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025


#### `__getitem__`
Fib实例虽然可以作用于for循环，但是还不能像list那样进行索引：

In [34]:
Fib()[5]

TypeError: 'Fib' object does not support indexing

要想像list那样按照下标取元素，需要实现`__getitem__`方法：

In [35]:
class Fib(object):
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a + b
        return a

In [36]:
f = Fib()
f[0]

1

In [37]:
f[1], f[2], f[4], f[5]

(1, 2, 5, 8)

list可以进行切片，Fib却报错：

In [38]:
list(range(100))[5:10]

[5, 6, 7, 8, 9]

`__getitem__`传入的参数可能是一个int，也可能是一个切片对象`slice`，这要进行判断：

In [39]:
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int):
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice):
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a, b = b, a + b
            return L

In [40]:
f = Fib()
f[0:5]

[1, 1, 2, 3, 5]

In [41]:
# 但是却没对step进行处理
f[:10:2]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

所以要正确实现一个`__getitem__`还是有很多工作要做的。总之通过上面的方法，我们定义的类表现的和Python自带的list、tuple、dict没什么区别，这完全归功于动态语言的“鸭子类型”，不需要强制继承某个接口。

#### `__getattr__`
正常情况下，我们调用类的方法和属性时，如果不存在，就会报错：

In [42]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'

In [43]:
s = Student()
print(s.name)

Michael


In [44]:
print(s.score)

AttributeError: 'Student' object has no attribute 'score'

要避免这个错误，除了可以加上一个`score`属性外，Python还有另一个机制，就是写一个`__getattr__`方法，动态返回一个属性：

In [45]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'
        
    def __getattr__(self, attr):
        if attr == 'score':
            return 99

In [46]:
s = Student()
s.name

'Michael'

In [47]:
s.score

99

返回函数也是可以的：

In [48]:
class Student(object):
    def __getattr__(self, attr):
        if attr == 'age':
            return lambda: 25

In [49]:
s = Student()
s.age()

25

只有在没有找到属性的情况下才会调用`__getattr__`，已有的不会调用。

实际上可以把一个类的所有属性和方法调用全部动态化处理。举个例子，现在很多网站都在高REST API，如果要写SDK，给每个URL对应的API都写一个方法，很麻烦，利用`__getattr__`，我们可以写出一个链式调用：

In [50]:
class Chain(object):
    
    def __init__(self, path=''):
        self._path = path
        
    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))
    
    def __str__(self):
        return self._path
    __repr__ = __str__

In [51]:
Chain().status.user.timeline.list

/status/user/timeline/list

这样，无论API怎么变，SDK都可以根据URL实现完全动态的调用，而且，不随API的增加而改变！

# 五、使用枚举类
当我们需要定义常量时，一个办法是用大写变量通过整数来定义，例如月份：

In [52]:
JAN = 1
FEB = 2
MAR = 3

这样做的好处是很简单，但缺点是类型为`int`，而且还是变量。更好的方法是为这样的枚举类型定义一个class类型，然后每个常量都是class的一个唯一实例：

In [53]:
from enum import Enum

Month = Enum('Month', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

In [55]:
for name, member in Month.__members__.items():
    print(name, '=>', member, ',', member.value)

Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
Apr => Month.Apr , 4
May => Month.May , 5
Jun => Month.Jun , 6
Jul => Month.Jul , 7
Aug => Month.Aug , 8
Sep => Month.Sep , 9
Oct => Month.Oct , 10
Nov => Month.Nov , 11
Dec => Month.Dec , 12


如果要精确的控制枚举类型，可以从`Enum`派生出自定义类：

In [56]:
from enum import Enum, unique

@unique
class Weekday(Enum):
    Sun = 0 # Sun的value被设定为0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6

In [59]:
day1 = Weekday.Mon
print(day1)

Weekday.Mon


In [60]:
print(Weekday.Tue)

Weekday.Tue


In [61]:
print(Weekday.Tue.value)

2


In [62]:
print(Weekday(1))

Weekday.Mon


In [63]:
Weekday(7)

ValueError: 7 is not a valid Weekday

In [64]:
for name, member in Weekday.__members__.items():
    print(name, '=>', member)

Sun => Weekday.Sun
Mon => Weekday.Mon
Tue => Weekday.Tue
Wed => Weekday.Wed
Thu => Weekday.Thu
Fri => Weekday.Fri
Sat => Weekday.Sat


## 练习

把`Student`的`gender`属性改造为枚举类型，可以避免使用字符串：

In [65]:
@unique
class Gender(Enum):
    Male = 0
    Female = 1

class Student(object):
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender
        
        
# 测试:
bart = Student('Bart', Gender.Male)
if bart.gender == Gender.Male:
    print('测试通过!')
else:
    print('测试失败!')

测试通过!


# 六、使用元类

### type()

动态语言和静态语言最大的不同就是函数和类的定义，不是编译时定义的，而是运行时动态创建的

In [66]:
class Hello(object):
    def hello(self, name='world'):
        print('Hello %s.' % name)

当Python解释器载入hello模块时，就会依次执行该模块所有的语句，结果就是动态创建出一个Hello的class对象：

In [67]:
h = Hello()
h.hello()

Hello world.


In [68]:
print(type(Hello))

<class 'type'>


In [69]:
print(type(h))

<class '__main__.Hello'>


我们说class的定义是运行时动态创建的，而创建class的方法就是使用`type()`函数。`type()`函数既可以返回一个对象的类型，又可以创建出新的类型，比如，我们可以通过`type()`函数创建出`Hello`类，而无需通过`class Hello(object)`来定义：

In [70]:
def fn(self, name='world'):
    print('Hello, %s' % name)

In [71]:
Hello = type('Hello', (object,), dict(hello=fn))

In [72]:
h = Hello()
h.hello()

Hello, world


In [73]:
print(type(Hello))

<class 'type'>


In [74]:
print(type(h))

<class '__main__.Hello'>


### metaclass

除了使用`type()`动态创建类以外，要控制类的创建行为还可以使用metaclass，元类。简单的解释就是
- 当我们定义了类以后，就可以根据这个类创建出实例，所以：先定义类后创建实例。但是如果我们想创建出类呢，就必须根据metaclass创建出类，所以：先定义metaclass，再创建类。

看一个简单的例子，这个metaclass可以给我们自定义的MyList增加一个add方法：

In [75]:
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

有了ListMetaclass，我们在定义类的时候还要指示使用ListMetaclass来定制类，传入关键字参数`metaclass`：

In [76]:
class MyList(list, metaclass=ListMetaclass):
    pass

当我们传入关键字参数`metaclass`时，魔术就生效了，它指示Python解释器在创建`MyList`时，要通过`ListMetaclass.__new__()`来创建，在此，我们可以修改类的定义，比如加上新的方法，然后返回修改后的定义：<br>
`__new__()`方法接收到的参数是：
1. 当前准备创建的类的对象
2. 类的名字
3. 类继承的父类集合
4. 类的方法集合

In [77]:
L = MyList()

In [78]:
L.add(1)

In [79]:
L

[1]

In [80]:
L.add(2)

In [81]:
L

[1, 2]

而普通的list则没有add方法

动态修改有什么意义？直接在`MyList`定义中写上`add()`方法不是更简单吗？正常情况下，确实应该直接写，通过metaclass修改纯属变态。

但是，总会遇到需要通过metaclass修改类定义的。ORM就是一个典型的例子。

ORM全称“Object Relational Mapping”，即对象-关系映射，就是把关系数据库的一行映射为一个对象，也就是一个类对应一个表，这样，写代码更简单，不用直接操作SQL语句。

要编写一个ORM框架，所有的类都只能动态定义，因为只有使用者才能根据表的结构定义出对应的类来。

让我们来尝试编写一个ORM框架。

其中，父类`Model`和属性`StringField`、`IntegerField`是由ORM框架提供的，剩下的魔术方法比如`save()`全部由metaclass自动完成。首先定义`Field`类：

In [84]:
class Field(object):
    
    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type
        
    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)

In [86]:
class StringField(Field):
    
    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')
        
class IntegerField(Field):
    
    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')

In [87]:
# 编写ModelMetaclass
class ModelMetaclass(type):
    
    def __new__(cls, name, bases, attrs):
        if name == 'Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings
        attrs['__table__'] = name
        return type.__new__(cls, name, bases, attrs)

In [91]:
# 基类Model
class Model(dict, metaclass=ModelMetaclass):
    
    def __init__(self, **kw):
        super(Model, self).__init__(**kw)
        
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)
            
    def __setattr__(self, key, value):
        self[key] = value
        
    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))

In [93]:
class User(Model):
    # 定义类的属性到列的映射
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')
    
# 创建一个实例
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
# 保存到数据库
u.save()

Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:username>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
SQL: insert into User (id,username,email,password) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']
